In [21]:
!pip install transformers accelerate datasets trl==0.11

In [22]:
!pip install --upgrade gcsfs fsspec

  Using cached gcsfs-2025.7.0-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
Using cached gcsfs-2025.7.0-py2.py3-none-any.whl (36 kB)
Using cached fsspec-2025.7.0-py3-none-any.whl (199 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.0
    Uninstalling gcsfs-2025.3.0:
      Successfully uninstalled gcsfs-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.7.0 which is incompatible.


In [29]:
!pip install fsspec==2025.3.0

  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.


In [23]:
!pip install gcsfs==2025.3.0 fsspec==2025.3.0




  Using cached gcsfs-2025.3.0-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached gcsfs-2025.3.0-py2.py3-none-any.whl (36 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.7.0
    Uninstalling gcsfs-2025.7.0:
      Successfully uninstalled gcsfs-2025.7.0


In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments
from trl import DPOTrainer, DPOConfig
from datasets import Dataset

In [2]:
import torch

In [17]:
data = {
    "prompt": [
        "What is the difference between civil and criminal law?",
        "Is data privacy protected by law?",
        "Can a minor enter into a contract?",
        "What are the key principles of intellectual property?",
        "Is freedom of speech unlimited?"
    ],
    "chosen": [
        "Civil law deals with disputes between individuals, while criminal law involves offenses against the state.",
        "Yes, many jurisdictions have laws that protect personal data, like GDPR in the EU.",
        "Generally no, unless the contract is for necessities or approved by a guardian.",
        "It includes protection of creations such as inventions, trademarks, and artistic works.",
        "No, it is subject to limitations such as hate speech, defamation, and national security concerns."
    ],
    "rejected": [
        "They're basically the same kind of law.",
        "Privacy isn't really something the law covers.",
        "Yes, anyone can sign anything.",
        "It's mostly about ideas and thoughts.",
        "Yes, people can say anything they want with no consequences."
    ]
}


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
dataset = Dataset.from_dict(data)

In [6]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
model.config.pad_token_id = tokenizer.pad_token_id

In [8]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model = model.to(device)

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id


In [10]:
from transformers import TrainingArguments

In [11]:
training_args = TrainingArguments(
    output_dir="./dpo_results",
    per_device_train_batch_size=2,
    learning_rate=5e-7,
    num_train_epochs=3,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_steps=500,
    remove_unused_columns=False,
    fp16=False
)


In [42]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [13]:
dpo_config = DPOConfig(
    output_dir="./dpo_results",
    per_device_train_batch_size=2,
    learning_rate=5e-7,
    num_train_epochs=3,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_steps=500,
    remove_unused_columns=False,
    beta=0.1,
    loss_type="sigmoid",
    fp16=False,
    bf16=False  # Added to explicitly disable bf16
)

In [14]:
trainer = DPOTrainer(
    model=model,
    args=dpo_config,  # Pass DPOConfig as args
    train_dataset=dataset,
    tokenizer=tokenizer,
)

print("DPO Trainer setup complete.")


Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:680: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:693: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Tokenizing train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

DPO Trainer setup complete.


/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:842: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [15]:
def generate_response(prompt_text):
    inputs = tokenizer(prompt_text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to device
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=20,
            num_return_sequences=1, # Sampling Strategies: Beam Search, Nucleus Sampling....
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text



In [16]:
# Example usage (before training, this will give baseline responses)
print("\nExample generation (before training):")
prompt = "Can a minor enter into a contract?"
response = generate_response(prompt)
print(f"Prompt: {prompt}")
print(f"Response: {response}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Example generation (before training):
Prompt: Can a minor enter into a contract?
Response: Can a minor enter into a contract?

Yes, you can enter into a contract.



In [ ]:
#Anser is yes. I think it should be no. Answer obtained by gpt-4: No, a minor generally cannot legally enter into a binding contract. there is a conflict.

In [18]:
print("\nExample generation (before training):")
prompt = "Is data privacy protected by law?"
response = generate_response(prompt)
print(f"Prompt: {prompt}")
print(f"Response: {response}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Example generation (before training):
Prompt: Is data privacy protected by law?
Response: Is data privacy protected by law?

The answer is yes. Data privacy is a fundamental right


In [ ]:
# this answer was not in chosen prompt. same example run on gpt-4 same answer recieved.